## First Question : scarp top 250 movie imbd with information

### import libraries

In [417]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

##### get main url

In [418]:
urlmain = 'https://www.imdb.com/chart/top/'


#### Send get request to url 

In [419]:
headers = {'Accept-Language': 'en-US',"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}

In [420]:
response = requests.get(url=urlmain , headers=headers)

In [421]:
print(response.status_code)

200


##### parse content

In [422]:
soup = BeautifulSoup(response.content, 'html.parser')

get link for each movies

In [423]:
links_250 = soup.find_all('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b85248f1-7 lhgKeb cli-title')

In [424]:
scrap_info_movies = []

In [425]:
import time
session = requests.Session()
for link in links_250[:3]: 
    a_element = link.find("a")
    get_att = a_element.get('href')
    movie_urls = f"https://www.imdb.com{get_att}"
    response_each_movie = requests.get(url=movie_urls, headers=headers)
    movies_soup = BeautifulSoup(response_each_movie.content, 'html.parser')
    titles = movies_soup.find('span', class_='sc-afe43def-1 fDTGTb').text

    ul_element = movies_soup.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'})
    years = None
    parental_guide = None
    runtime = None
    for li_element in ul_element.find_all('li', {'class': 'ipc-inline-list__item'}):
        ainner_element = li_element.find('a', {'class': 'ipc-link ipc-link--baseAlt ipc-link--inherit-color'})
        if ainner_element:
            href = ainner_element.get('href')
            text = ainner_element.text
            if 'releaseinfo' in href:
                years = text
            elif 'parentalguide' in href:
                parental_guide = text
        else:
            runtime = li_element.text
    genre_list = movies_soup.find_all('a', {'class': 'ipc-chip ipc-chip--on-baseAlt'})
    genre = [genre.find('span', class_='ipc-chip__text').text for genre in genre_list]
    #director
    ul_crew = movies_soup.find('ul' , class_= 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt')
    directors = None
    stars = None
    writers = None
    for li_element in ul_crew.find_all('li' , {'class': 'ipc-inline-list__item'}):
        ainner_crew = li_element.find('a', {'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
        if ainner_crew:
            href = ainner_crew.get('href')
            text = ainner_crew.text
            if 'tt_ov_wr' in href:
                writers = text
            elif 'tt_ov_st' in href:
                stars = text
            elif 'tt_ov_dr' in href:
                directors = li_element.text
    scrap_info_movies.append({'title': titles,'runtime':runtime, 'parental_guide' : parental_guide, 'genre': genre, 'stars': stars, 'directors': directors, 'writers': writers})
    #time.sleep(2)

session.close()


In [426]:
top_250 = pd.DataFrame(scrap_info_movies)

In [427]:
top_250.head(50)

,title,runtime,parental_guide,genre,stars,directors,writers
0,The Shawshank Redemption,2h 22m,R,[Drama],None,Frank Darabont,None
1,The Godfather,2h 55m,R,"[Crime, Drama]",None,Francis Ford Coppola,None
2,The Dark Knight,2h 32m,PG-13,"[Action, Crime, Drama]",None,Christopher Nolan,None


In [428]:
"""     directore_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    directore = [directore.get_text() for directore in directore_list]
    writer_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    writer = 
    star_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    star = 
    gross_us_canada =   """

"     directore_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    directore = [directore.get_text() for directore in directore_list]\n    writer_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    writer = \n    star_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    star = \n    gross_us_canada =   "